In [1]:
import json
import random
from os import listdir
from os.path import isfile, join
from IPython.display import clear_output

In [2]:
class Sample:
    help_text = '\n'.join([
        'Type 2 integers in order of preference.',
        'For example, type "15 or 23".',
    ])
  
    def __init__(self, prompt, truth, random, sentiment, rank=None):
        self.prompt = prompt
        self.truth = truth
        self.random = random
        self.sentiment = sentiment
        self.rank = rank
    
    @staticmethod
    def load_file(path):
        '''
        Create a list of Samples from a json file.
        The json should be either a single dict or a list of dicts.
        Each dict must contain the keys "prompt", "truth", "random", and "sentiment".
        '''
        with open(path) as f:
            j = json.load(f)
        if isinstance(j, dict):
            j = [j]
        return [Sample.load_dict(d) for d in j]
  
    @staticmethod
    def load_dict(d):
        return Sample(d['prompt'], d['truth'], d['random'], d['sentiment'], d.get('rank'))
  
    def evaluate(self):
        def print_header(text=None, c='='):
            if not text:
                print(c * 62)
                return
            left = max(30 - len(text) // 2, 10)
            right = max(30 - (len(text) + 1) // 2, 10)
            print(c * left, text, c * right)
            
        clear_output()
        print_header('Prompt')
        print(self.prompt)
        completions = [self.truth, self.random, self.sentiment]
        print_header('Truth', c='-')
        print(self.truth)
        print_header('A', c='-')
        version = 0
        if random.random() > 0.5:
            print(self.random)
            print_header('B', c='-')
            print(self.sentiment)
        else:
            version = 1
            print(self.sentiment)
            print_header('B', c='-')
            print(self.random)
        
    
        while True:
            print_header(c='-')
            res = input(f'Rank samples ([h]elp, [q]uit) >>> ')
            if res.startswith('h'):
                print(self.help_text)
                continue
            if res.startswith('q'):
                return None
            if len(res) != 2:
                print('Invalid rank, try again.')
            elif version == 0:
                self.rank = {'random':res[0], 'sentiment':res[1]}
                break
            else:
                self.rank = {'random':res[1], 'sentiment':res[0]}
                break
  
    def __str__(self):
        j = {
            'prompt': self.prompt,
            'truth': self.truth,
            'random': self.random,
            'sentiment': self.sentiment,
        }
        if self.rank:
            j['rank'] = self.rank
        return json.dumps(j, indent=4)

In [3]:
def evaluate_movie(src, dest, test_count):
    with open(src, "r") as f:
        j = json.load(f)
        evaluations = ""
        for test in j:
            t_id = test['id']
            t_prompt = test['prefix'][-200:]
            t_true = test['truth'][0:200]
            t_random = test['random'][0:200]
            t_sentiment = test['sentiment'][0:200]
            s = Sample(t_prompt, t_true, t_random, t_sentiment) 
            s.evaluate()
            evaluations += str(t_id) + "," + s.rank['random'] + "," + s.rank['sentiment'] + '\n'
    with open(dest, "a+") as f:
        f.write(evaluations)

def evaluate_entire_dir(srcpath, destpath, outfile):
    evaluations = ""
    dest = join(destpath, outfile)
    test_count = 3
    for f in listdir(srcpath):
        src = join(srcpath, f)
        if isfile(src):
            evaluate_movie(src, dest, test_count)

In [ ]:
evaluate_entire_dir('./all_samples/Matt/', './evaluations/', 'evals_matt.txt')

=========================== Prompt ===========================
 The spilled coins start SLIDING across the floor all by themselves, as if magnetically drawn to THE CHIMNEY -- WHOOSH! -- SUCKED right up into it by some VACUUM FORCE! THE WOMAN What's happening?!!! 
---------------------------- Truth ---------------------------
KA-CHINNNG! A STEEL COLLAR SHOOTS DOWN FROM THE LIGHT, CLAMPING 
AROUND HER NECK! A CABLE SNAPS OUT FROM IT AND WHIPS UPWARD, 
PULLING HER INTO THE LIGHT -- She manages to GRAB a doorknob and DANGLES 
------------------------------ A -----------------------------
AST! THE CHIMNEY LATER, the CHIMNEY LOCKS ON THE CHIMNEY, YOU GUYS. Jake pulls the coin BANGING, SHOTGUNS his finger in the WINDOW, SICKS it, AND BANGS it right into the CHIMNEY's BOWDING BOTTLE! THE 
------------------------------ B -----------------------------
CKIE This is a MAN! I don't need to see this! You can't help yourself and I can't help myself. Don't you know why I'm so scared? I could be the o